# Voorbeeld fragility curve piping

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from toolbox_continu_inzicht import Config, DataAdapter
from toolbox_continu_inzicht.fragility_curves import (
    FragilityCurvesPiping,
    CombineFragilityCurvesWeightedSum,
    CombineFragilityCurvesIndependent,
)

In [ ]:
test_path = Path.cwd() / "../../../"
dataset_path = test_path / "src/fragility_curves/fragility_curves_piping/data_sets/"

# Lees originele config
config = Config(config_path=dataset_path / "test_fragility_multiple_curves_piping.yaml")
config.lees_config()

# Corrigeer rootdir
config.global_variables["rootdir"] = str(dataset_path)

# Pas output aan van csv naar dataframe
config.data_adapters["fragility_curves"]["type"] = "python"
del config.data_adapters["fragility_curves"]["file"]
del config.data_adapters["fragility_curves"]["sep"]

data_adapter = DataAdapter(config=config)

In [ ]:
```yaml
GlobalVariables:
    rootdir: "data_sets"
    moments: [-24,0,24,48]

    fragility_curve_piping:
        type: ci_postgresql_fragilitycurves
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        failuremechanism: "STBI"

    fragility_curve_overtopping:
        type: ci_postgresql_fragilitycurves
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
        failuremechanism: "GEKB"

    fragility_curves:
        type: ci_postgresql_fragilitycurves_table
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    fragility_curves_csv:
        type: csv
        file: "test.csv"
```

In [ ]:
plt.close("all")

fragility_curves_overtopping = CombineFragilityCurves(data_adapter=data_adapter)
fragility_curves_overtopping.run(
    input=["fragility_curve_piping", "fragility_curve_overtopping"],
    output="fragility_curves_csv",
)

In [ ]:
# Combineer meerdere vakken tot 1 curve
comb_fc = CombineFragilityCurvesIndependent(data_adapter=data_adapter)
comb_fc.run(input=section_names, output="fragility_curves")

plt.close("all")
colx = "waterlevels"
coly = "failure_probability"
fig, ax = plt.subplots(figsize=(5, 3))
for section_name in section_names:
    data_adapter.input(section_name).plot(colx, coly, ax=ax, label=section_name)
comb_fc.df_out.plot(colx, coly, ax=ax, label="Traject curve")
ax.set_title("Vak en traject curves")
fig.tight_layout()